In [291]:
import sqlite3
import requests
import os
import cv2
from datetime import date
import urllib
import re
import urllib.request
import urllib.parse
# pip install opencv-python

In [292]:
def Imgurl(word):
    rep_list = []
    #Simulate browser, need to use browser information and target url
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        "referer": "https://image.baidu.com"
    }
    #Encrypt Chinese keywords into garbled characters that the browser can recognize
    content= urllib.parse.quote(word,encoding='utf-8')
    #Circulate 4 times from 30 to 121 according to the law of pn, the interval is 30
    for num in range(30,121,30):
        gsm = hex(num)[2:]         #Convert decimal number num to hexadecimal number and take the last two digits
        url = 'https://image.baidu.com/search/acjson?tn=resultjson_com&ipn=rj&ct=201326592&is=&fp=result&queryWord='+content+'&cl=2&lm=-1&ie=utf-8&oe=utf-8&adpicid=&st=&z=&ic=&word='+content+'&s=&se=&tab=&width=&height=&face=&istype=&qc=&nc=&fr=&pn='+str(num)+'&rn=30&gsm='+ gsm +'&1521707235798='    #根据规律每次循环生成正确的请求地址
        req = urllib.request.Request(url=url,headers=header)   #Get request object
        page = urllib.request.urlopen(req).read()     #Request and read return information
        try:              #If the returned information encounters characters that are not in the range of utf-8, skip
            response = page.decode('utf-8')                       #Decode the information returned
            imgpattern = re.compile(r'"thumbURL":"(.*?)\.jpg')    #regex
            rsp_data = re.findall(imgpattern, response)           
            rep_list += rsp_data
        except UnicodeDecodeError:
            pass
    return rep_list




In [300]:
folder_path = '/Users/liuhaoyue/Desktop/image_downloader/' # my local location
if os.path.exists(folder_path) == False:  
    os.makedirs(folder_path)
i = 1
online_url = []
local_url = []
for index, item in enumerate(Imgurl('山')): # ‘山’ means mountains in Chinese
    if i <= 100:
        online_url.append(item)
        img_name = folder_path + str(index + 1) +'.png'
        local_url.append(img_name)
#         print(item)
        html = requests.get(item)
        request = html.request
        a = session.send(request)
        with open(img_name, 'wb') as file:  
                file.write(a.content)
                file.flush()
                file.close()
        print('Loading %d-th pictures' %(index+1))
        i += 1
    else:
        break
print('Finish loading pictures')


Loading 1-th pictures
Loading 2-th pictures
Loading 3-th pictures
Loading 4-th pictures
Loading 5-th pictures
Loading 6-th pictures
Loading 7-th pictures
Loading 8-th pictures
Loading 9-th pictures
Loading 10-th pictures
Loading 11-th pictures
Loading 12-th pictures
Loading 13-th pictures
Loading 14-th pictures
Loading 15-th pictures
Loading 16-th pictures
Loading 17-th pictures
Loading 18-th pictures
Loading 19-th pictures
Loading 20-th pictures
Loading 21-th pictures
Loading 22-th pictures
Loading 23-th pictures
Loading 24-th pictures
Loading 25-th pictures
Loading 26-th pictures
Loading 27-th pictures
Loading 28-th pictures
Loading 29-th pictures
Loading 30-th pictures
Loading 31-th pictures
Loading 32-th pictures
Loading 33-th pictures
Loading 34-th pictures
Loading 35-th pictures
Loading 36-th pictures
Loading 37-th pictures
Loading 38-th pictures
Loading 39-th pictures
Loading 40-th pictures
Loading 41-th pictures
Loading 42-th pictures
Loading 43-th pictures
Loading 44-th pictur

In [301]:
conn = sqlite3.connect(':memory:') 
c = conn.cursor()
sql = """
CREATE TABLE myimages (
    id INTEGER PRIMARY KEY autoincrement,
    image_name VARCHAR(80),
    image_url VARCHAR(80),
    image_shape VARCHAR(80),
    download_datetime DATE,
    image_location VARCHAR(80),
    biodata BLOB
);
"""
c.execute(sql)

In [304]:
for index, item in enumerate(local_url):
    name = 'Picture %d' %(index+1)
    image_url = online_url[index]
    im = cv2.imread(item) #get the image shape
    shape = str(im.shape)
    download_date = str(date.today()) #get the download time
    local_location = item # the location of loaded pictures
    picture = open(item, 'rb').read()
    sql = '''INSERT INTO myimages (image_name, image_url, image_shape, download_datetime,image_location, biodata) VALUES (?,?,?,?,?,?)'''
    c.execute(sql,(name,image_url,shape, download_date,local_location,picture))
    conn.commit()
    

In [307]:
c.execute(
    """
    SELECT *
    FROM myimages 
    """
)
c.fetchall()[0]

(1,
 'Picture 1',
 'https://ss0.bdstatic.com/70cFuHSh_Q1YnxGkpoWK1HF6hhy/it/u=2542415979,1523368429&fm=26&gp=0',
 '(332, 500, 3)',
 '2020-10-18',
 '/Users/liuhaoyue/Desktop/image_downloader/1.png',
 b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x01L\x01\xf4\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x02\x02\x03\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x04\x02\x05\x00\x01\x06\x07\x08\xff\xc4\x00?\x10\x00\x02\x01\x03\x02\x04\x04\x04\x04\x05\x03\x03\x04\x02\x03\x00\x01\x02\x03\x00\x04\x11\x12!\x05\x131A"Qaq\x06\x142\x81\x91\xa1\xb1\xc1\x15#B\xd1\xf0\x07R\xe1\x16b\xf1$3CrS\x824s\x92\xff\xc4\x00\x1a\x01\x00\x03\x01\x01\x01\x01\x00\x00\x00\